In [1]:
results, err = compute_train_performance(optimal_distThresh, image_dir = TEST_DIRECTORY returnResults=True)

In [2]:
#Process and image
def process_img(img_file, shape = (256,256)):
    img = imread(img_file, plugin='tifffile', as_gray = True)
    img = transform.resize(img, shape)
    return(img)
    

def watershed_count(img_file, distance_threshold = 0.7, plotResult = True):
    test_img = cv2.imread(img_file)
    gray = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,distance_threshold*dist_transform.max(),255,0)
    #imshow(sure_fg)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    markers = cv2.watershed(test_img,markers)
    test_img[markers == -1] = [255,0,0]
    if plotResult:
        plt.imshow(markers, cmap = 'jet')
    
    ncells = markers.max()-1
    return(ncells)

def RMSE(c, c_hat):
    return np.sqrt(np.mean(np.power(c-c_hat, 2)))

########### UNET MODEL ###############3
#Compute dice coeficient used in loss function
def dice_coef(y_true, y_pred):
    #Dice coeficient parameter
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

#Loss function
def dice_coef_loss(y_true, y_pred):
    return(-dice_coef(y_true, y_pred))

def get_unet(img_size = (256,256,1)):
    inputs = Input(img_size)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef, 'accuracy'])
    #model.summary()
    return(model) 



In [3]:
unet = get_unet()
unet.load_weights('./pretrainedUnet/unetCellCnt_weights.08--1.99.hdf5')

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
# get known cell counts
def compute_train_performance(x, image_dir = TRAIN_DIRECTORY, returnResults = False):
    img_files = [f for f in os.listdir(image_dir) if f.endswith('.TIF')]
    nfiles = len(img_files)
    if not os.path.exists(image_dir +'masks'):
        os.makedirs(image_dir +'masks')
        for i in range(nfiles):          
            # make masks...
            f = img_files[i]
            img_file = image_dir + f
            img = process_img(img_file)
            pred = unet.predict(img.reshape(1,256,256,1))
            pred = pred.round(0).reshape((256,256))
            pred = pred*255
            scipy.misc.toimage(pred).save(image_dir + 'masks/'+ (img_files[i])[:-4] + '.png')
    
    
    labels =pd.read_csv(image_dir + 'labels.csv')
    mask_files = [f for f in os.listdir(image_dir+'masks') if f.endswith('.png')]
    results = pd.DataFrame({'image_file':labels.image_name, 'c':np.zeros((nfiles)), 'c_hat':np.zeros((nfiles))})

    for j in range(nfiles):     
        #print(mask_files[j])
        mask_name = (labels.image_name[j])[:-4] + '.png'
        mask_file = image_dir + 'masks/' + mask_name
        c = labels.at[j, 'count']
        #print(c)
        c_hat = watershed_count(image_dir + '/masks/' +mask_name,  
                                distance_threshold=x,
                                plotResult = False)
        #print(c_hat)
        results.at[j,'c'] = c
        results.at[j,'c_hat'] = c_hat
    if returnResults:
        print(results)
        return (results, -RMSE(results.c, results.c_hat))
    else:
        return(-RMSE(results.c, results.c_hat))  


In [17]:
compute_train_performance(0.7)

-52.21791778211766

In [18]:
pbounds = {'x': (0.01, .99)}
optimizer = BayesianOptimization(
    f = compute_train_performance,
    pbounds = pbounds,
    verbose = 2,
    random_state = 1)

In [19]:
optimizer.maximize(init_points = 4, n_iter = 80)
optimal_distThresh = optimizer.max['params']['x']

|   iter    |  target   |     x     |
-------------------------------------
|  1        | -38.79    |  0.4187   |
|  2        | -52.81    |  0.7159   |
|  3        | -45.06    |  0.01011  |
|  4        | -39.83    |  0.3063   |
|  5        | -38.48    |  0.3749   |
|  6        | -38.55    |  0.3927   |
|  7        | -38.57    |  0.3576   |
|  8        | -38.52    |  0.3702   |
|  9        | -40.52    |  0.4694   |
|  10       | -38.95    |  0.3405   |
|  11       | -42.45    |  0.2214   |
|  12       | -38.52    |  0.383    |
|  13       | -57.82    |  0.99     |
|  14       | -38.66    |  0.4048   |
|  15       | -38.46    |  0.3775   |
|  16       | -39.24    |  0.4371   |
|  17       | -38.49    |  0.3788   |
|  18       | -39.33    |  0.3254   |
|  19       | -38.79    |  0.3498   |
|  20       | -38.96    |  0.4272   |
|  21       | -38.56    |  0.3637   |
|  22       | -40.64    |  0.2816   |
|  23       | -38.54    |  0.388    |
|  24       | -42.77    |  0.5165   |
|  25       

In [22]:
results, err = compute_train_performance(0.7, image_dir = TEST_DIRECTORY, returnResults=True)

                  image_file      c  c_hat
0       A01_C1_F1_s02_w1.TIF    1.0    1.0
1       A01_C1_F1_s03_w2.TIF    1.0    1.0
2       A01_C1_F1_s04_w2.TIF    1.0    1.0
3       A01_C1_F1_s05_w1.TIF    1.0    1.0
4       A01_C1_F1_s07_w2.TIF    1.0    1.0
5       A01_C1_F1_s11_w1.TIF    1.0    1.0
6       A01_C1_F1_s13_w1.TIF    1.0    1.0
7       A01_C1_F1_s24_w1.TIF    1.0    1.0
8       A01_C1_F1_s25_w1.TIF    1.0    1.0
9       A02_C5_F1_s18_w1.TIF    5.0    5.0
10      A02_C5_F1_s19_w2.TIF    5.0    5.0
11     A03_C10_F1_s01_w2.TIF   10.0    9.0
12     A03_C10_F1_s04_w2.TIF   10.0    8.0
13     A03_C10_F1_s07_w1.TIF   10.0   10.0
14     A03_C10_F1_s10_w1.TIF   10.0   10.0
15     A03_C10_F1_s13_w2.TIF   10.0    7.0
16     A03_C10_F1_s23_w1.TIF   10.0   10.0
17     A03_C10_F1_s23_w2.TIF   10.0    9.0
18     A04_C14_F1_s11_w2.TIF   14.0   12.0
19     A04_C14_F1_s19_w2.TIF   14.0    2.0
20     A04_C14_F1_s23_w1.TIF   14.0   12.0
21     A04_C14_F1_s25_w2.TIF   14.0    4.0
22     A05_

In [23]:
err

-52.34757635268323

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(test_img,markers)
test_img[markers == -1] = [255,0,0]
imshow(markers)
markers.max()-1

In [ ]:
img_file = 'test.png'
distance_threshold = 0.1
plotResult = True
test_img = cv2.imread(img_file)
gray = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,distance_threshold*dist_transform.max(),255,0)
#imshow(sure_fg)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(test_img,markers)
test_img[markers == -1] = [255,0,0]
if plotResult:
    plt.imshow(markers, cmap = 'jet')

ncells = markers.max()-1

In [ ]:
ncells

In [ ]:
markers = cv2.watershed(test_img,markers)
img[markers == -1] = [255,0,0]

In [ ]:
imshow(markers)